In [ ]:
import pandas as pd
import nannyml as nml

reference, analysis, analysis_target = nml.datasets.load_synthetic_binary_classification_dataset()
reference['y_pred'] = reference['y_pred_proba'].map(lambda p: int(p >= 0.8))
analysis['y_pred'] = analysis['y_pred_proba'].map(lambda p: int(p >= 0.8))

reference.head(3)

In [ ]:
def showrst(df):
    print(df.to_markdown(tablefmt="grid"))

In [ ]:
data = pd.concat([reference, analysis.set_index('identifier').join(analysis_target.set_index('identifier'), on='identifier', rsuffix='_r')], ignore_index=True).reset_index(drop=True)
data.loc[data['partition'] == 'analysis'].head(3)

In [ ]:
metadata = nml.extract_metadata(reference, model_type=nml.ModelType.CLASSIFICATION_BINARY)
metadata.target_column_name = 'work_home_actual'
metadata.to_df()

In [ ]:
performance_calculator = nml.PerformanceCalculator(model_metadata=metadata, metrics=['roc_auc', 'recall'], chunk_size=5000)
performance_calculator.fit(reference_data=reference)

In [ ]:
realized_performance = performance_calculator.calculate(data)

In [ ]:
showrst(realized_performance.data.head(3))

In [ ]:
fig = realized_performance.plot(kind='performance', metric='roc_auc')
fig.write_image(file="../_static/performance_calculation_roc_auc.svg", engine="kaleido")
fig.show()

In [ ]:
fig = realized_performance.plot(kind='performance', metric='recall')
fig.show()
